In [13]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [2]:
# -----------------------------------------------------
# OPEN FILE
# -----------------------------------------------------
filename="shape_coleta.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

gdf = gpd.read_file(arquivo)
gdf_t=gdf[['setor','geometry']].copy()

gdf_t['n_dias']=gdf['dias'].apply(lambda x: len(eval(x))) #criando uma coluna com numerodias

filename="shape_bairros.gpkg"
arquivo = os.path.join(path, filename)
gdf_b = gpd.read_file(arquivo)
gdf_b.rename(columns={"NM_BAIRRO": "Bairro"}, inplace=True)


In [3]:
gdf_b.head(3)

,Bairro,geometry
0,Marco,"MULTIPOLYGON (((-48.45072 -1.43336, -48.45066 ..."
1,Reduto,"MULTIPOLYGON (((-48.48838 -1.44583, -48.48836 ..."
2,Batista Campos,"MULTIPOLYGON (((-48.4884 -1.46606, -48.48916 -..."


In [18]:
gdf_t = gdf_t.to_crs(gdf_b.crs)


gdf_b = gdf_b.to_crs(epsg=3857)
gdf_t = gdf_t.to_crs(epsg=3857)
print(gdf_b.crs)
print(gdf_t.crs)

EPSG:3857
EPSG:3857


In [5]:
# bairro_select= gdf_b[gdf_b['Bairro'] == 'Condor']

# setores_no_bairro= gdf_t[gdf_t.intersects(bairro_select.iloc[0].geometry)]


In [26]:
bairro_select= gdf_b[gdf_b['Bairro'] == 'Condor']

setores_no_bairro= gdf_t[gdf_t.intersects(bairro_select.iloc[0].geometry)].copy()

# # Seleciona o bairro desejado
bairro_geom = bairro_select.iloc[0].geometry

# Inicializa uma lista para armazenar os percentuais
percentuais = []

# Loop sobre cada setor
for setor_geom in setores_no_bairro.geometry:
    # Calcula a área de interseção com o bairro
    intersecao = setor_geom.intersection(bairro_geom)
    # Calcula o percentual coberto do setor
    percentual = (intersecao.area / setor_geom.area) * 100
    # Adiciona à lista
    percentuais.append(percentual)

# Adiciona a lista como nova coluna no GeoDataFrame
setores_no_bairro['percentual_coberto'] = np.round(percentuais,2)

# Mostrar o setor e o percentual
setores_no_bairro[['setor', 'percentual_coberto', 'n_dias']]


,setor,percentual_coberto,n_dias
1,1305,65.92,6
4,1302,0.02,6
5,1303,65.79,6
6,1306,2.97,6
30,1304,48.08,6
